In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy.stats import norm, linregress
import seaborn as sb
from nltk.stem import PorterStemmer

In [2]:
csv_path_1 = "Resources/tmdb_5000_movies.csv"
csv_path_2 = "Resources/tmdb_5000_credits.csv"
df_credits = pd.read_csv(csv_path_2, low_memory=False)
credits_json = ['cast', 'crew']
for i in credits_json:
    df_credits[i] = df_credits[i].apply(json.loads)

In [3]:
# defination cells
# safely load the json files, if it is not, return nan
def load_safety(temp_json, index):
    result = temp_json
    try:
        return result[index]['name']
    except:
        return np.nan
# return the names in a list
def list_name(keywords):
    return [x['name'] for x in keywords]
# return the directors name in a list
def director_name(crew):
    result = [x['name'] for x in crew if x['job'] == 'Director']
    try:
        return result[0]
    except:
        return np.nan

In [5]:
# input the dataframe from csv (tmdb movies and credits)
df_movies = pd.read_csv(csv_path_1)
# turn the release_date into time type
df_movies['release_date'] = pd.to_datetime(df_movies['release_date']).apply(lambda x: x.date())
# turn all potential json column into real json formate
json_column = ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']
for i in json_column:
    df_movies[i] = df_movies[i].apply(json.loads)
# safely load country, language, first actor, second actor, third actor and fourth actor
json_1 = ['country', 'language']
json_2 = ['production_countries', 'spoken_languages']
json_app = [0,0]
json_clean_first = ['first_actor', 'second_actor', 'third_actor', 'fourth_actor']
json_clean_second = ['cast', 'cast', 'cast', 'cast']
json_apply = [1,2,3,4]
for i in range(2):
    df_movies[json_1[i]] = df_movies[json_2[i]].apply(lambda x: load_safety(x, json_app[i]))
for i in range(4):
    df_movies[json_clean_first[i]] = df_credits[json_clean_second[i]]\
                                    .apply(lambda x: load_safety(x, json_apply[i]))
# safely load director and put keywords, production_companies and genres in a list
for i in range(3):
    df_movies[json_column[i]] = df_movies[json_column[i]].apply(list_name)
df_movies['director'] = df_credits['crew'].apply(director_name)
# output an initial cleanup data
df_movies.to_csv('Resources/intial_clean_data.csv', index = False)
print("The shape of the initial_clean_data table: " + str(df_movies.shape))

The shape of the initial_clean_data table: (4803, 27)


In [6]:
type_and_missing = pd.DataFrame(df_movies.dtypes).T.rename(index = {0: 'type'})
df_null_sum = pd.DataFrame(df_movies.isnull().sum())
type_and_missing = type_and_missing.append(df_null_sum.T.rename(index = {0: 'count of missing cells'}))
type_and_missing.to_csv('Output/missing_count.csv', index = False)
type_and_missing

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,title,vote_average,vote_count,country,language,first_actor,second_actor,third_actor,fourth_actor,director
type,int64,object,object,int64,object,object,object,object,float64,object,...,object,float64,int64,object,object,object,object,object,object,object
count of missing cells,0,0,3091,0,0,0,0,3,0,0,...,0,0,0,174,86,53,63,93,169,30


In [8]:
df_missing = df_movies.isnull().sum(axis = 0).reset_index()
df_missing = df_missing.rename(columns = {'index': 'column_name', 0: 'total_missing'})
df_missing['missing_percentage'] = df_missing['total_missing'].divide(df_movies.shape[0]/100)
df_missing.sort_values('missing_percentage', ascending = False).reset_index(drop = True).head()

,column_name,total_missing,missing_percentage
0,homepage,3091,64.355611
1,tagline,844,17.572351
2,country,174,3.622736
3,fourth_actor,169,3.518634
4,third_actor,93,1.936290


In [40]:
df_copy_movies = df_movies.copy()
ps = PorterStemmer()
keyword_stem = {}
for cells in df_copy_movies['keywords']:
    if pd.isna(cells).any():
        continue
    for i in cells:
        i = i.lower()
        i_stem = ps.stem(i)
        if i_stem in keyword_stem.keys():
            keyword_stem[i_stem][1] += 1
            if i not in keyword_stem[i_stem][0]:
                keyword_stem[i_stem][0].append(i)
        else:
            keyword_stem[i_stem] = [[i],1]
df_stem = pd.DataFrame(keyword_stem).T.rename(columns = {0: 'un_stem list', 1: 'total numbers'})\
          .sort_values('total numbers', ascending = False)
df_stem.to_csv('Output/stem_count.csv', index = False)
df_stem.head()

,un_stem list,total numbers
woman director,[woman director],324
independent film,[independent film],318
duringcreditssting,[duringcreditsstinger],307
murder,"[murder, murderer]",197
based on novel,[based on novel],197


,un_stem list,total numbers
woman director,[woman director],324
independent film,[independent film],318
duringcreditssting,[duringcreditsstinger],307
murder,"[murder, murderer]",197
based on novel,[based on novel],197
